# Ridge Regression Recovery

This paper https://arxiv.org/abs/1610.03045 gives an iteratvie method for solving the Ridge regression problem up to $\epsilon$ precision using random projections.  The key idea is that a relationship between the primal and dual can be exploited to reduce dimensionality on both the sample size and number of features.

In [13]:
import numpy as np
from numpy.linalg import norm, eig
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.kernel_approximation import RBFSampler
import scipy
import matplotlib.pyplot as plt

rng = np.random.RandomState(0)

The random projections used in the paper come from a family called _Hessian Sketches_.  Details are omitted here but for now it is enough just to apply a Gaussian matrix as the Hessian sketch of suitable dimension.  From https://arxiv.org/abs/1501.06195 it is suggested that if a Gaussian matrix has sufficiently large dimension, namely at least $c d_n$ where $d_n$ is some index between 1 and $n$ denoting the index of the largest eigenvalue which is upper bounded by the ctrical radius of the data.  It is also referred to in https://arxiv.org/abs/1411.0347 as the _Gaussian Width_ which for $n \times d$ data can be bounded as $O(d)$.  I assume this is the same as for the JLT condition that we need something like $O(d/\epsilon^2 \text{poly log}(d))$. So for now we focus on two sketches $\Pi$ and $R$ which will reduce the input size.

In [2]:
def Hessian_Sketch(num_rows, num_cols):
    '''Returns a (num_rows)*(num_cols) matrix which is the 
    Hessian sketch.  Future functionality to extend from 
    Gaussian to ROS sketches which are faster to apply.'''
    return np.random.randn(num_rows, num_cols)

In [96]:
def iterative_primal_dual_sketch(data, targets,
                                 offset, tolerance,
                                max_iterations=10):
    '''Perform the vanilla iterative_primal_dual_sketching.'''
    X = data
    y = targets
    _lambda = offset 
    n, d = X.shape
    is_converged = False
    
    
    
    # sketch parameters.  Do need to satisfy certain bounds
    p = d//2
    m = n//2
    # initialise sketches nb. I have switched notation from the R in paper
    feature_sketch = Hessian_Sketch(d, p)
    
    sample_sketch = Hessian_Sketch(m,n) 
    # same as in paper for Pi but needs to be transposed for  right multiplication
    
    
   
    
    
   
    X_R = np.dot(X,feature_sketch) # just renaming to avoid unnecessary compute
    double_sketch = (0.5*n)*np.dot(sample_sketch, X_R)
    
    ### parameters for iteration
    w_hat = np.zeros(shape=(d,1))
    #u = np.random.randn(p,1)
    
    #print("X_R shape {}".format(X_R.shape))
    #print("X_Rz shape {}".format(np.dot(X_R,z).shape))
    #print("y shape {}".format(y.shape))
    #print("X w_hat shape {}".format(np.dot(X, w_hat).shape))
    
    
    for t in range(max_iterations):
        k = 0
        z = np.ravel(np.ones(shape=(p,1)))
        #z = np.ravel(np.random.randn(p,1))
        w_hat = np.ravel(w_hat)
        while not is_converged:
            print("iteration on (t,k) ({},{})".format(t,k))
            print("norm(z) = {}".format(norm(z,2)))
            #print("Start loop shape {}".format(z.shape))
            # this is just problem set up
            
            print("X_R shape {}".format(X_R.shape))
            print("y shape {}".format(y.shape))
            print("X shape {}".format(X.shape))
            print("w_hat shape {}".format(w_hat.shape))
            temp1 = (1/n)*(np.dot(X_R.T, y) - np.dot(X_R.T, np.dot(X,w_hat)) - np.dot(X_R.T, np.dot(X_R,z)))
            print("feature ksetch shape {}".format(feature_sketch.shape))
            print("z shape {}".format(z.shape))
            temp2 = _lambda*np.dot(feature_sketch.T, w_hat) + _lambda*z
            temp3 = np.ravel(temp1 - temp2) # remove column dimension
            print("temp1 shape {}".format(temp1.shape))
            print("temp2 shape {}".format(temp2.shape))
            print("temp3 shape {}".format(temp3[:,None].shape))
            
            
            #####################
            #def linear_function(u):
            #    sketched_vector = np.dot(double_sketch,u)
            #    #print("Sketched vector shape {}".format(sketched_vector.shape))
            #    regulariser = 0.5*_lambda*norm(u,2)**2
            #    inner_product = np.dot(temp3,u)
            #    term1 = norm(sketched_vector,2)**2
            #    return term1 + regulariser - inner_product

            #def grad(u):
             #   return 2*np.dot(double_sketch.T, np.dot(double_sketch, u))\
              #          +_lambda*u + temp3
            ###################
            
            print("Covariance shape {}".format(np.dot(double_sketch.T,double_sketch).shape))
            print("temp3 shape {}".format(temp3[:,None].shape))
            z_hat = np.linalg.lstsq(2*np.dot(double_sketch.T,double_sketch) \
                                    +_lambda*np.eye(p),temp3[:,None])[0]
            print("z_hat shape {}".format(z_hat.shape))
            
            
            
            #result = scipy.optimize.minimize(fun=linear_function, x0=z,
                                          #method="Newton-CG", jac=grad, tol=1e-5)
            #print("norm of Gradient at norm(z) = {} is {}".format(norm(z), norm(grad(z),2)))
            z_new = z[:,None] + z_hat 
            print("z_new shape {}".format(z_new.shape))
            print("norm(z_new) = {}".format(norm(z_new)))
            #print("Type result.x {}".format(type(result.x)))
            #print("Result.x shape {}".format(result.x.shape))
            #print("End loop shape {}".format(z_new.shape))
            #print("End loop shape {}".format(z.shape))
            #print(norm(z_new - z,2))
            if norm(z_new - z,2) < tolerance:
                z = z_new
                print("Residual: {}".format(norm(z_new - z,2)))
                print("Exiting loop.")
                is_converged = True
            else:
                z = z_new # flip the labelling so can just plug z in above code
                k += 1
                print("Looping again.")
            #print("norm of Gradient at norm(z) = {}".format(norm(z)))
            
        
        
        is_converged = False # reset so the loop is entered next time over    
        # Update dual  
        print(np.dot(X, w_hat).shape)
        print(z.shape)
        print(X_R.shape)
        print(w_hat.shape)
        print("y shape {}".format(y.shape))
        alpha_dual = y[:, None] - np.dot(X, w_hat[:,None]) - np.dot(X_R, z)
        print('Alpha shape {}'.format(alpha_dual.shape))
        
        # Update primal
        w_hat = (1/(n*_lambda))*np.dot(X.T, alpha_dual)
        print(w_hat.shape)
    return w_hat, alpha_dual
            
            

In [97]:
w, alpha = iterative_primal_dual_sketch(X, y, offset=1e-5, tolerance=1e-6)

iteration on (t,k) (0,0)
norm(z) = 22.360679774997898
X_R shape (5000, 500)
y shape (5000,)
X shape (5000, 1000)
w_hat shape (1000,)
feature ksetch shape (1000, 500)
z shape (500,)
temp1 shape (500,)
temp2 shape (500,)
temp3 shape (500, 1)
Covariance shape (500, 500)
temp3 shape (500, 1)
z_hat shape (500, 1)
z_new shape (500, 1)
norm(z_new) = 22.36067977499778
Residual: 0.0
Exiting loop.
(5000,)
(500, 1)
(5000, 500)
(1000,)
y shape (5000,)
Alpha shape (5000, 1)
(1000, 1)
iteration on (t,k) (1,0)
norm(z) = 22.360679774997898
X_R shape (5000, 500)
y shape (5000,)
X shape (5000, 1000)
w_hat shape (1000,)
feature ksetch shape (1000, 500)
z shape (500,)
temp1 shape (500,)
temp2 shape (500,)
temp3 shape (500, 1)
Covariance shape (500, 500)
temp3 shape (500, 1)
z_hat shape (500, 1)
z_new shape (500, 1)
norm(z_new) = 22.360679788172764
Looping again.
iteration on (t,k) (1,1)
norm(z) = 22.360679788172764
X_R shape (5000, 500)
y shape (5000,)
X shape (5000, 1000)
w_hat shape (1000,)
feature kset

LinAlgError: Incompatible dimensions

### Test on real data

In [5]:
from sklearn.datasets import make_regression

In [6]:
X,y, coef = make_regression(n_samples=5000, n_features=1000, n_informative=1000, n_targets=1, coef=True)